## Section 1: Setting Up Credentials for CJA Data Queries

In this first step of our journey through Adobe's Customer Journey Analytics (CJA), we establish the essential connection credentials. We'll cover two primary methods for querying data:
1. **Via the Reporting API**: Using the `cjapy` library, a powerful tool created by [Julien Piccini](https://github.com/pitchmuc/cjapy), to directly interface with the CJA Reporting API.
2. **Via the PostgreSQL Connector**: A method preferred by SQL enthusiasts, utilizing SQL queries through the CJA BI Extension for data retrieval.

### Preparing for JWT Authentication
For this exercise, we've pre-configured the JWT authentication setup for you. However, for your own projects, you'll need to set up an Adobe Console API project for each organization you wish to access. Here are the steps you'd typically follow to create an API client in the Adobe Developer Console:

1. Go to [console.adobe.io](https://console.adobe.io/) and ensure you're logged into the correct Adobe organization.
2. Start a **new project** and **add the CJA API** to it.
3. **Generate a keypair**; this action downloads a `config.zip` file containing your private key. Securely store this file.
4. After generating the keypair, select the product profiles required for your API access and save your configured API.
5. If you're also using the Adobe Experience Platform API, add it to your project without generating a new keypair.
6. Locate the **Service Account (JWT)** section under **CREDENTIALS** and either download the service account JSON file or create one manually with your credentials.

### Reporting API Setup with cjapy
For querying the CJA Reporting API, we utilize the `cjapy` library. Ensure you have a `python_config.json` file prepared with the following credentials:
- `org_id`: Your Adobe Org ID.
- `client_id`: The client ID provided by Adobe.
- `secret`: Your client secret.
- `tech_id`: The technical account ID.
- `pathToKey`: The file path to your private key.

This initial setup ensures we can securely and efficiently access CJA for our data analysis tasks.

In [ ]:
# Thanks to Julien Piccini for his hard work on the cjapy library!
import cjapy
'''
    For using cjapy, you'll need a config file that contains the following:
    {
        "org_id": "YOURORGID@AdobeOrg",
        "client_id": "YOURCLIENTID",
        "secret": "YOURCLIENTSECRET",
        "tech_id": "YOURTECHID@techacct.adobe.com",
        "pathToKey":"path_to_private.key"
    }
'''

# Instantiate cjapy
cjapy.importConfigFile("python_config.json")

cja = cjapy.CJA()

## Section 2: Exploring Data Views in CJA

Once our credentials are set, the next logical step is to identify the Data Views available to us in Customer Journey Analytics (CJA). Data Views are essentially lenses through which we can examine our collected data, each configured with specific dimensions and metrics to suit various analysis needs.

### Identifying Available Data Views
Understanding the Data Views accessible to us is crucial for tailoring our data queries accurately. This process involves listing all Data Views and selecting the one most relevant to our analysis goals.

Follow these steps to list and select your desired Data View:
1. Retrieve a list of all available Data Views using the CJA API.
2. Review the list to identify the Data View that best matches your analysis criteria.
3. Note the ID of your chosen Data View for subsequent queries.

In [ ]:
# Retrieve a list of all available Data Views
data_views = cja.getDataViews()

# Print each Data View's name and ID for easy identification
print("Available Data Views:")
for index, row in data_views.iterrows():
    print(f"Data View Name: {row['name']}, Data View ID: {row['id']}")

# Example: Selecting a specific Data View by its ID for further queries
data_view_id = "dv_62ba17d5a5d7845496f5fb4d"  # Replace with the ID of your chosen Data View
print(f"\nSelected Data View ID for Analysis: {data_view_id}")

## Section 3: Discovering Metrics and Dimensions

A critical step in our data analysis journey is understanding the metrics and dimensions available within our chosen Data View in CJA. This knowledge allows us to tailor our data queries more effectively, ensuring we extract the most relevant insights.

### Exploring Available Metrics
Metrics, such as page views, orders, or time spent, quantify user interactions and behaviors. Identifying the available metrics helps in shaping our analysis and choosing the right parameters for our reports.

### Identifying Available Dimensions
Dimensions provide context to our metrics, such as product names, user IDs, or geographic locations. By exploring available dimensions, we can segment our data in meaningful ways, offering deeper analytical insights.

Let's use the `cjapy` library to list the metrics and dimensions available in our selected Data View, setting the stage for targeted and informed data queries.

In [ ]:
# Grab all available metrics for the specified Data View
metrics = cja.getMetrics(data_view_id)

# Print the name and ID of each metric
print("Available Metrics:")
for index, row in metrics.iterrows():
    print(f"Metric: {row['name']}, ID: {row['id']}")

# Grab all available dimensions for the specified Data View
dimensions = cja.getDimensions(data_view_id)

# Print the name and ID of each dimension
print("\nAvailable Dimensions:")
for index, row in dimensions.iterrows():
    print(f"Dimension: {row['name']}, ID: {row['id']}")

## Section 4: Retrieving Orders by Product Name

In this section, we'll delve into extracting specific data from CJA — namely, the number of orders by product name. This example serves as a foundation for querying different metrics and dimensions. Feel encouraged to explore additional data ranges, metrics, or dimensions to suit your analytical needs.

### Crafting the Report Request
Our goal is to construct a report request that targets a specific dimension (`product_name`) and metric (`orders`) over a defined date range. This process involves:
1. Selecting the desired dimension and metric for the report.
2. Defining the date range for our data query.
3. Utilizing the `cjapy` library to construct and send the report request to CJA.
4. Extracting and displaying the report data in a readable format.

This approach allows us to analyze product performance, guiding strategic decisions based on order volume.

In [ ]:
from cjapy import RequestCreator
import pandas as pd

# Define the dimension and metric of interest
dimension = "variables/product_name"
metric = "metrics/orders"
dateRange = "2024-01-01T00:00:00.000/2024-01-17T00:00:00.000"

# Initialize the report request with the specified Data View ID
myRequest = RequestCreator()
myRequest.setDataViewId(data_view_id)
myRequest.setDimension(dimension)
myRequest.addMetric(metric)
myRequest.addGlobalFilter(dateRange)

# Execute the report request and retrieve the data
myReport = cja.getReport(myRequest)
report_df = myReport.dataframe[[dimension, metric]]

# Display the first few rows of the report dataframe to verify the data
print(report_df.head(10))

## Section 5: Querying CJA with SQL via the BI Extension

SQL enthusiasts have a robust alternative to the `cjapy` library for querying data from Adobe's Customer Journey Analytics (CJA) — the BI Extension. This section illuminates the process of leveraging SQL for data retrieval, highlighting its familiarity and adaptability for comprehensive data analysis.

### Adobe Query Service Credentials Overview

Adobe Experience Platform Query Service provides two types of credentials to facilitate secure connections to external clients:

- **Expiring Credentials**: Offer a quick setup for temporary access with a default 24-hour validity. Ideal for short-term projects or exploratory data analysis.
- **Non-Expiring Credentials**: Suited for enduring connections, necessitating configuration via the Adobe Admin Console for ongoing data access.

**Important Note**: The credentials panel may not be immediately accessible and might require coordination with your admin or Adobe account team for activation. For a detailed guide on generating and managing these credentials, refer to the [official Adobe documentation](https://experienceleague.adobe.com/docs/experience-platform/query/ui/credentials.html?lang=en).

### Setting Up SQL Connection

For our lab, we've preconfigured the SQL connection using credentials from the Adobe Developer Console, enabling a seamless and secure setup for executing SQL queries within the CJA BI Extension and added these to the Python config file.

### Performing SQL Queries

Our focus will be on executing an SQL query to summarize orders by product name, showcasing SQL's robust capabilities for data manipulation and analysis. This practical example aims to demonstrate the versatility of SQL in extracting and analyzing CJA data, providing a powerful alternative to traditional API-based queries.

In [ ]:
from sqlalchemy import create_engine
import json

# Load credentials from the previously prepared JSON file
with open('python_config.json', 'r') as file:
    config = json.load(file)

# Setup database connection details
username = config["PostgreSQL_username"]
password = config["PostgreSQL_password"]
host = 'acxpevangelist.platform-query.adobe.io'
port = 80
dbname = 'prod:cja'

# Create the database engine for the connection
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{dbname}')

# Define and execute the SQL query
query = """
    SELECT product_name, SUM(orders) AS orders, SUM(product_views) AS product_views
    FROM adobe_store
    GROUP BY product_name
    ORDER BY orders DESC
"""

# Execute the query and read the results into a DataFrame
df = pd.read_sql(query, engine)

# Display the query results
print(df.head())